# Chapter 10 - Petrological thermodynamics of liquid and solid phases

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## The equilibrium state

### Approaching the eutectic phase diagram

In [ ]:
class R_DMC_Calibration:
    def __init__(self):
        self.nc = 2
        self.np = 300
        self.Tref = 1350
        self.Tmin = 950
        self.Tmax = 1900
        self.T_degC = np.linspace(self.Tmin, self.Tmax, self.np)
        self.Cref = np.asarray([0.75, 0.25])
        self.Pref = 1
        self.Pmin = 0
        self.Pmax = 7
        self.P_GPa = np.linspace(self.Pmin, self.Pmax, self.np)
        # data R_DMC_CalibrationTwoComponent
        self.T0 = np.asarray([1780, 976])  # set pure component melting points T_m^i at P=0
        self.A = np.asarray([45, 107.4])  # set coeff. for linear P-dependence of T_m^i [K/GPa]
        self.B = np.asarray([-2.0, -2.83])  # set coeff. for quadratic P-dependence of T_m^i [K/GPa^2]
        self.L = np.asarray([600e3, 450e3])  # set latent heat of pure components L^i [J/kg]
        self.r = np.asarray([70, 40])  # set coeff. for T-dependence of distribution coefficients K^i
        self.Tm_P_mode = 'simonslaw'
        self.dS = 1.0
        self.K_T_mode = 'inverse_exp'
        self.CompStr = {'dunite', 'morb'}  # {'dunite~', 'morb~'}


class VAR:
    def __init__(self, np, nc, pref, tref):
        self.C = np.zeros(np, nc)
        self.C[:, 1] = np.linspace(0, 1, np)
        self.C[:, 0] = 1.0 - self.C[:, 1]
        self.P = np.ones(nc) * pref
        self.T = np.ones(nc) * tref
        self.f = np.zeros(nc)
        self.Cl = None
        self.Cs = None

In [ ]:
def TM(P, r_dmc_calib):
    # compute P - dependence of component melting points
    Tm = np.zeros((P.shape[0], r_dmc_calib.nc))
    if r_dmc_calib.Tm_P_mode is 'quadratic':
        r_dmc_calib.Tm  = np.asarray([r_dmc_calib.T0[i] + r_dmc_calib.A[i] * P + r_dmc_calib.B[i] * P**2
                                      for i in np.arange(0, r_dmc_calib.nc)])
        # safeguard: continue melting point with linear slope above Pmax
        Pmax = 6
        T0 = np.asarray([r_dmc_calib.T0[i] + r_dmc_calib.A[i] * Pmax + r_dmc_calib.B[i] * Pmax**2
                         for i in np.arange(r_dmc_calib.nc)])
        dTdP = np.asarray([(r_dmc_calib.A[i] * Pmax + r_dmc_calib.B[i] * Pmax**2) -
                           (r_dmc_calib.A[i] * (Pmax - 0.01) + r_dmc_calib.B[i] * (Pmax - 0.01)**2)
                           for i in np.arange(r_dmc_calib.nc)]) / 0.01
        ind = P > Pmax
        Tm[ind, :] = np.asarray([T0 + dTdP * (P[ind] - Pmax) for _ in np.arange(r_dmc_calib.nc)])
    elif r_dmc_calib.Tm_P_mode is 'simonslaw':  # check below
        # Parameterization as in Rudge et al(2011)
        Tm[:, :] = np.asarray([r_dmc_calib.T0[i] * np.power(1.0 + P / r_dmc_calib.A[i], 1.0/r_dmc_calib.B[i])
                               for i in np.arange(r_dmc_calib.nc)])
    return Tm

In [ ]:
def K_(T, r_dmc_calib):
    # ** *compute T, P - dependence of equilibrium partition coefficients
    L = (T + 273.15) * r_dmc_calib.dS
    K = np.zeros((T.shape[0], r_dmc_calib.nc))
    if r_dmc_calib.K_T_mode is 'inverse_exp':
        # Parameterization after Rudge, Bercovici, & Spiegelman(2010)
        for i in np.arange(r_dmc_calib.nc):
            K[:, i] = np.exp(L[:, i] / r_dmc_calib.r[i] * (1.0 / (T + 273.15) - 1.0/(r_dmc_calib.Tm[:, i] + 273.15)))
    elif r_dmc_calib.K_T_mode is 'linear_exp':
        # Linearised exponential dependence
        for i in np.arange(r_dmc_calib.nc):
            K[:, i] = np.exp(-r_dmc_calib.r[i] * (T - r_dmc_calib.Tm[:, i]))
    return L, K

In [ ]:
def Tsolidus(var, r_dmc_calib):
    # ***** subroutine to compute solidus temperature at given bulk composition
    # *** exclude invalid compositions
    ii = np.sum(var.C[:, np.sum(var.C, 1) > 1], 2) <= 1
    # ** *get P - dependent pure - component    melting Temperature
    Tm = TM(var.P, r_dmc_calib)
    # ** *set starting guess for Tsol:
    Tsol = max(min(min(Tm)), min(max(max(Tm)), np.sum(var.C * r_dmc_calib.Tm, 2)))
    # ** *get T - dependent partition coefficients Ki
    PAR = K_(Tsol, r_dmc_calib)
    # ** *get residual for sum(ci_b / Ki) = 1
    r = np.sum(var.C / r_dmc_calib.K, 2) - 1
    rnorm = 1  # initialize residual norm for iterations
    n = 0  # initialize iteration count
    rnorm_tol = 1e-10  # tolerance for Newton residual
    its_tol = 500  # maximum number of iterations
    eps_T = 5  # temperatur perturbation for finite differencing, degrees

    while rnorm > rnorm_tol:  # iterate down to full accuracy

        # ** *compute partition coefficients Ki at T + eps_T
        _, K = K_(Tsol + eps_T, r_dmc_calib)
        # ** *get residual at
        T + eps_T
        rp = np.sum(var.C / K, 2) - 1

        # ** *compute partition coefficients Ki at T - eps_T
        _, K = K_(Tsol - eps_T, r_dmc_calib)
        # ** *get residual at T - eps_T
        rm = np.sum(var.C / K, 2) - 1

        # ** *finite difference
        drdT = (r(T + eps_T) - r(T - eps_T)) / 2 / eps_T
        drdT = (rp - rm)/ 2. / eps_T

        # ** *apply Newton correction to current guess of Tsol
        Tsol[ii] -= r[ii] / drdT[ii]

        # ** *compute partition coefficients Ki at Tsol
        _, K = K_(Tsol, r_dmc_calib)

        # ** *compute residual at Tsol
        r = np.sum(var.C / K, 2) - 1

        # ** *compute Newton residual norm
        rnorm = np.linalg.norm(r[ii], 2) / np.sqrt(len(r[ii]))

        n = n + 1  # update iteration count

        if n == its_tol:
            raise('!!! Newton solver for solidus T has not converged after ' + str(its_tol) + ' iterations !!!')

    return Tsol

In [ ]:
def Tliquidus(var, r_dmc_calib):
    # subroutine to compute liquidus temperature at given bulk composition
    # ** *exclude invalid compositions
    ii = np.sum(var.C[:, np.sum(var.C, 1) > 1], 2) <= 1
    # ** *get P - dependent pure component melting T
    Tm = TM(var.P, r_dmc_calib)

    # ** *set starting guess for Tliq
    Tliq = max(min(min(Tm)), min(max(max(Tm)), sum(VAR.C * Tm, 2)))

    # ** *get T - dependent partition coefficients Ki
    _, K = K_(Tliq, r_dmc_calib)

    # ** *get residual for sum(ci_b * Ki) = 1
    r = np.sum(var.C * K, 2) - 1.0

    rnorm = 1  # initialize residual norm for iterations
    n = 0  # initialize iteration count
    rnorm_tol = 1e-10  # tolerance for Newton residual
    its_tol = 500  # maximum number o iterations
    eps_T = 1.0  # temperature perturbation for finite differencing, degrees

    while rnorm > rnorm_tol:  # iterate down to full accuracy
        # ** *compute partition coefficients Ki at T + eps
        _, K = K_(Tliq + eps_T, r_dmc_calib)
        # ** *get residual at T + eps_T
        rp = np.sum(var.C * r_dmc_calib.K, 2) - 1

        # ** *compute partition coefficients Ki at T - eps_T
        _, K = K_(Tliq - eps_T, r_dmc_calib)
        # ** *get residual at T - eps_T
        rm = np.sum(var.C * r_dmc_calib.K, 2) - 1

        # ** *compute difference
        drdT = (r(T + eps_T) - r(T - eps_T)) / 2 / eps_T
        drdT = (rp - rm) / 2. / eps_T

        # ** *apply Newton correction to Tliq
        Tliq[ii] = Tliq[ii] - r[ii] / drdT[ii]

        # ** *compute partition coefficients Ki at Tliq
        _, K = K_(Tliq, r_dmc_calib)
        # ** *compute residual at Tliq
        r = np.sum(var.C * K, 2) - 1

        # ** *compute Newton residual norm
        rnorm = np.linalg.norm(r[ii], 2) / np.sqrt(len(r[ii]))

        n = n + 1  # update iteration count

        if n == its_tol:
            raise Exception('Newton solver for liquidus T has not converged '
                            'after ' + str(its_tol) + ' iterations !!!')

    return Tliq

In [ ]:
def Equilibrium(var, r_dmc_calib):
    
    # subroutine to compute equilibrium melt fraction and phase
    # compositions at given bulk composition, pressure and temperature

    # get Tsol, Tliq at C = [C1,C2,C3]
    Tsol = Tsolidus(VAR, r_dmc_calib)
    Tliq = Tliquidus(VAR, r_dmc_calib)

    # get P-dependent pure component melting T
    Tm = TM(var.P, r_dmc_calib)

    # get T-dependent partition coefficients Ki
    _, K = K_(max(Tsol,min(Tliq, var.T)), r_dmc_calib)

    # compute residual of unity sum of components
    ff = np.repmat(var.f, 1, r_dmc_calib.nc)
    r = np.sum(var.C / (ff + (1-ff) * K), 2) - np.sum(var.C / (ff/K+(1-ff)), 2)

    rnorm = 1  # initialize residual norm for iterations
    n = 0  # initialize iteration count
    rnorm_tol = 1e-10  # tolerance for Newton residual
    its_tol = 100  # maximum number of iterations

    if var.T <= Tsol:
        var.f[:] = 0  # no melt below sC1idus
    elif var.T >= Tliq:
        var.f[:] = 1  # fully molten above liquidus
    else:
        while rnorm > rnorm_tol:  # Newton iteration

            # compute analytic derivative of residual dr/df
            dr_df = - np.sum(var.C * (1.0 - K)/(ff+(1-ff)*K)**2, 2) + np.sum(var.C * (1/K-1)/(ff/K+(1-ff))**2, 2)

            # apply Newton correction to f
            a = 1
            while ((var.f - a * r/dr_df) < -1e-16).any() or (var.f - a*r/dr_df > 1-1e-16).any():
                a = a/2
                if a < 1e-6:
                    raise Exception('R_DMC module: Newton solver for equilibrium melt fraction did not converge '
                                    'with step size {}!'.format(2*a))
            var.f = var.f - a * r/dr_df

            # compute residual of unity sum of components
            ff = np.repmat(var.f, 1, r_dmc_calib.nc)
            r = np.sum(var.C/(ff+(1-ff)*K), 2) - np.sum(var.C/(ff/K+(1-ff)), 2)

            # get non-linear residual norm
            rnorm = np.linalg.norm(r, 2) / np.sqrt(len(r))

            n = n+1  # update iteration count
            if n==its_tol:
                raise Exception('Newton solver for equilibrium f did not converge'
                                ' after ' + str(its_tol) + ' iterations !!!')

    # get C_s^i, C_l^i as functions of K^i, C^i and f, safeguard bounds
    var.Cl = max(0.0, min(1.0, var.C/(ff + (1-ff)*K)))
    VAR.Cs = max(0.0, min(1.0, var.C/(ff/K + (1-ff))))

    # safeguard bounds on melt fraction against numerical leaks
    f = max(0, min(1, var.f))

    return var.T, f

In [ ]:
def R_DMC_Equilibrium(var, r_dmc_calib, eq_type):
    """
    main routine to compute various quantities depending input 'type'
    :param var:
    :param r_dmc_calib:
    :param type:
    :return:
    """

    if var.f is None:
        var.f = np.zeros_like(var.T)
    if var.C is None:
        var.C = np.zeros((len(var.T), r_dmc_calib.nc))
    if var.Cl is None:
        var.Cl = var.C
    if var.Cs is None:
        var.Cs = var.C

    # compute partition coefficients at P,T - this part is not used in "IdealSolutionVsEutectic.m"
    # if type is 'K':
    #    Tm = Tm(var.P, r_dmc_calib)
    #    L, K = K(var.T, r_dmc_calib)
    # compute Tsol, Tliq at P,C^i
    if eq_type is 'T':
        Tsol = Tsolidus(var, r_dmc_calib)
        Tliq = Tliquidus(var, r_dmc_calib)
        return Tsol, Tliq
    # compute equilibrium f, C_s^i, C_l^i fraction at P,T,C^i
    elif eq_type is 'E':
        T, f = Equilibrium(var, r_dmc_calib)
        return T, f
    else:
        raise Exception('type {} not recognized'.format(eq_type))

In [ ]:
r_dmc_calib = R_DMC_Calibration()

r2 = np.asarray([70, 50, 30, 10])
cI = len(r2) + 1
cp = 1200
sf = 0.3
H = np.linspace(-r_dmc_calib.L[0], r_dmc_calib.L[0] + cp * (r_dmc_calib.T0[0] - r_dmc_calib.T0[1]), 1000)

T = np.linspace(r_dmc_calib.T0[0], r_dmc_calib.T0[1])
X = (T - r_dmc_calib.T0[1]) / (r_dmc_calib.T0[0] - r_dmc_calib.T0[1])
C_ = 1 - ((1 - sf) * X + sf * X ** 3)

var = VAR()

for i, r2i in enumerate(r2):
    r_dmc_calib.r[1] = r2i
    _, r_dmc_eq = R_DMC_Equilibrium(var, r_dmc_calib, 'T')
    plt.plot(var.C[:, 1], r_dmc_eq.Tsol, '-', linewidth=2, color=[1-i/cI, 1-i/cI, 1-i/cI])
    plt.plot(var.C[:, 2], r_dmc_eq.Tliq, '-', linewidth=2, color=[1-i/cI, 1-i/cI, 1-i/cI])
plt.show()


### Application to a three-pseudo-component system

### Linearising the two-component phase diagram